# MUM 2023-24 Graf Obliczeniowy

## Graf obliczeń

* Jest to graf skierowany acykliczny DAG
  * Każdy wierzchołek oznacza wykonanie pewnej funkcji na wartościach wierzchołków połączonych krawędziami wchodzącymi
  * Wszystkie wierzchołki bez krawędzi wchodzących traktujemy jako input grafu.
* Do tej pory rozważaliśmy funkcje operujące na wektorach i zwracające wektory
  * W praktyce będziemy jednak używali operacji, które przyjmują na wejściu wielowymiarowe tablice float'ów i zwracają jedną tablicę. Wymiary poszczególnych tablic mogą się różnić.
* Celem grafu obliczeń jest zareprezentowanie (skomplikowanej) funkcji wielokrotnie złożonej w sposób, który umożliwi jej automatyczne różniczkowanie
  * Graf będzie składał się z __modelu__ oraz __funkcji kosztu__
  * wejście grafu wykorzystamy do wprowadzenia __danych treningowych__ oraz __parametrów modelu__. Naszym celem będzie minimalizowanie funkcji kosztu poprzez gradientową optymalizację parametrów modelu.
<img style="float: right;" src="ml_figures/Graf_obliczen_graf1.png" width=400>

* Umiemy różniczkować funkcje złożone (mnożenie jakobianów), ale jak policzyć jakobian, skoro funkcja operuje na tablicach, a nie wektorach?
  * Otóż musielibyśmy najpierw rozwinąć wszystkie inputy i skleić je w jeden wektor, który funkcja $f$ przekształca w rozwinięty output
  * Z reguły to podejście jest niepraktyczne, więc będziemy starali się działać sprytniej.
<img style="float: right;" src="ml_figures/Graf_obliczen_graf2.png" width=650>


## Cel algorytmu

* Mamy pewien input (dane, parametry modelu), który chcemy podać do grafu. Na jego podstawie obliczamy wartość funkcji kosztu, reprezentowaną przez wierzchołek $L$. Dla każdego wierzchołka typu input chcemy obliczyć gradient funkcji kosztu po tym wierzchołku, w punkcie wyznaczonym przez podane na input wartości liczbowe.
* Gradienty będziemy oznaczali literą $\delta$. W efekcie chcemy więc, aby w każdym wierzchołku typu input obliczona została tablica liczb $\delta$ - jej wymiary są zgodne z podaną na input tablicą wartości liczbowych.
* Oczywiście możemy policzyć gradienty również po innych wierzchołkach, które leżą na drodze pomiędzy inputami a wierzchołkiem $L$ - tak właśnie zrobimy, ponieważ to umożliwi nam obliczenie gradientów inputów.
* Musimy wiedzieć, w jakim punkcie liczymy gradienty. W wypadku wierzchołków typu input są to po prostu odpowiednie wartości podawane na input. Natomiast w wypadku pozostałych wierzchołków musimy na podstawie inputu obliczyć i zapamiętać ich wartości liczbowe.
* Podsumowując:
  1. chcemy obliczyć i zapamiętać wartości liczbowe wszystkich wierzchołków, począwszy od inputu, skończywszy na funkcji kosztu
  2. chcemy w tych wierzchołkach obliczyć wartości gradientu funkcji kosztu, oznaczamy je symbolem $\delta$
  3. chcemy zwrócić gradienty policzone w wierzchołkach typu input, a w szczególności w wierzchołkach opisujących parametry modelu
* będziemy uczyć model gradientowo

## Przykład

* Regresja liniowa. Naszym zadaniem jest dopasowanie prostej do $N$-elementowego zbioru punktów treningowych $(x_n, y_n)$.
<img style="float: left;" src="ml_figures/Graf_obliczen_linreg2.png" width=450>

  * Równanie prostej to $\widehat{y} = ax + b$.
  * Niech $\mathbf{x}$ oznacza wektor złożony ze wszystkich $x_n$, natomiast $\mathbf{y}$ wektor wszystkich $y_n$.
  * Funkcję kosztu definiuje się jako $L(a,b) = \dfrac{1}{N}\sum_{n=1}^N (ax_n+b-y_n)^2$.

<img style="float: right;" src="ml_figures/Graf_obliczen_linreg.png" width=400>

  * Dwuelementowy wektor $\theta=(a,b)$ to parametry modelu.
  * Dla tak postawionego problemu graf obliczeń wygląda następująco:

## Backpropagation
* Funkcję kosztu będziemy z reguły oznaczali literą $L$ (_loss function_).
* __Node__ weźmy dowolny wierzchołek, który opisuje funkcję $f$. Niech jej input to ciąg tablic $X_1, \ldots, X_K$, a output to tablica $Y$. Wierzchołek w grafie musi implementować dwie metody.
* __Forward pass__ to po prostu implementacja funkcji $f$. Metoda przyjmuje $X_1, \ldots, X_K$ i zwraca $Y$
* __Backward pass__ metoda przyjmuje $X_1, \ldots, X_K$ oraz $\delta_Y$ i zwraca $\delta_{X_1}, \ldots, \delta_{X_K}$. Tablica $\delta_Y$ ma takie same wymiary jak $Y$ i przechowuje gradient $L$ po $Y$. Metoda ma obliczyć i zwrócić gradient $L$ po inputach. Jak to zrobić?
  * Niech $x$ będzie pewnym elementem tablicy $X_k, k\in\{1,\ldots,K\}$. Trzeba sprawdzić, jaki ma on wpływ na wszystkie elementy outputu (odpowiednia pochodna cząstkowa funkcji $f$), a następnie przemnożyć je przez odpowiadające elementy $\delta_Y$ i zsumować - w ten sposób policzymy, jak łącznie wpływa on na $L$ za pośrednictwem $f$. Uzyskaną liczbę wpisujemy w odpowiednie miejsce tablicy $\delta_{X_k}$.
  * Procedurę powtarzamy dla wszystkich elementów $X_k$, a następnie dla wszystkich $k$.
* Wartości $X_1, \ldots, X_K$ są potrzebne, ponieważ pochodna cząstkowa jest funkcją i musi być liczona w punkcie.
  * Dla różnych wartości $X_1, \ldots, X_K$ wpływ tego samego elementu inputu na pewien element outputu może się znacznie różnić, bo funkcja $f$ w różnych miejscach swojej dziedziny ma inne liniowe przybliżenie.

#### Przykład: numpy.multiply
* Niech $f$ oznacza funkcję, która przyjmuje dwie tablice (tego samego rozmiaru) $X_1, X_2$ i zwraca ich iloczyn element-wise $Y$.
  * Niech $x_1$ oznacza pewien element $X_1$, natomiast $x_2, y, \delta_y$ oznaczają elementy odpowiednio $X_2, Y, \delta_Y$, które mają te same współrzędne, co $x_1$ w $X_1$.
  * Z definicji $f$ zachodzi $y = x_1 x_2$. Ponadto $x_1, x_2$ nie mają wpływu na pozostałe elementy $Y$. W takim razie
$$ \delta_{x_1} = \dfrac{\partial L}{\partial y}(y)\cdot \dfrac{\partial y}{\partial x_1}(x_1, x_2) = \delta_y \cdot x_2 $$
$$ \delta_{x_2} = \dfrac{\partial L}{\partial y}(y)\cdot \dfrac{\partial y}{\partial x_2}(x_1, x_2) = \delta_y \cdot x_1 $$

* Jeśli np. tablice $X_1, X_2, Y$ mają rozmiar $(4,3)$, to jakobian $f$ ma rozmiar $(12,24)$, przy czym jedynie 24 z 288 elementów jakobianu jest niezerowych (bo na każdy $y$ mają niezerowy wpływ tylko dwie rzeczy).
  * Widzimy, że znacznie łatwiej jest zaimplementować backward pass bez mnożenia explicite $\delta_Y$ przez jakobian.
* __Graph__
  1. Wprowadź do grafu obliczeń zbiór treningowy oraz bieżące parametry modelu.
  2. Forward pass: przelicz po kolei wszystkie funkcje (wierzchołki) w grafie, aż do momentu uzyskania wartości funkcji kosztu. Zapamiętaj w każdym pośrednim wierzchołku obliczone wartości - wraz z inputem grafu będą one potrzebne do policzenia pochodnej.
  3. Ustaw $\delta_L = 1$. (dlaczego?)
  4. Dla każdego wierzchołka $W$:
    1. Niech $V_1, \ldots, V_K$ będą wszystkimi wierzchołkami, których inputem jest $W$.
    2. Dla każdego wierzchołka $V_k$ wykonaj krok 4. rekurencyjnie - oblicz i zapisz $\delta_{V_k}$.
    3. Dla każdego wierzchołka $V_k$ wywołaj jego metodę backward pass (użyj zapisanych wartości $\delta_{V_k}$ oraz warości inputów zapamiętanych w kroku 2.). Zapamiętaj wszystkie $\delta_W^k$.
    4. Zsumuj $k$ zapamiętanych tablic $\delta_W^k$, zapisz sumę jako $\delta_W$. (dlaczego suma?)

![graf3](../ml_figures/Graf_obliczen_graf3.png)

* __Training loop__
  * Uczenie przebiega w pętli.
  * Wykonujemy pełny forward pass, a następnie backward pass.
  * Delty parametrów modelu (gradient funkcji kosztu) przekazujemy do optimizera.
  * Optimizer oblicza nowe wartości parametrów, które użyjemy w następnym kroku.
  * Uczenie kończymy np. po określonej liczbie kroków lub po osiągnięciu satysfakcjonującej wartości funkcji kosztu.

### autograd
* w praktyce wszystkie gradienty są obliczane automatycznie
* wystarcza do tego zestaw podstawowych informacji oraz implementacja reguły łańcuchowej
* pakiet `torch.autograd` (czy innej implementacji)